In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import LSTM
import numpy as np
from tensorflow.keras.metrics import *
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/50
8/8 [==============================] - 1s 79ms/step - loss: 1.3680 - accuracy: 0.5148 - precision: 0.4789 - recall: 0.3972 - auc: 0.5080
Epoch 2/50
8/8 [==============================] - 0s 58ms/step - loss: 1.3082 - accuracy: 0.4984 - precision: 0.4698 - recall: 0.5444 - auc: 0.5035
Epoch 3/50
8/8 [==============================] - 1s 65ms/step - loss: 1.2415 - accuracy: 0.5257 - precision: 0.4937 - recall: 0.4556 - auc: 0.5248
Epoch 4/50
8/8 [==============================] - 1s 81ms/step - loss: 1.1979 - accuracy: 0.5093 - precision: 0.4730 - recall: 0.4089 - auc: 0.5112
Epoch 5/50
8/8 [==============================] - 1s 72ms/step - loss: 1.1618 - accuracy: 0.4786 - precision: 0.4341 - recall: 0.3692 - auc: 0.4766
Epoch 6/50
8/8 [==============================] - 1s 76ms/step - loss: 1.1030 - accuracy: 0.5356 - precision: 0.5080 - recall: 0.2967 - auc: 0.5448
Epoch 7/50
8/8 [==============================] - 1s 76ms/step - loss: 1.0856 - accuracy: 0.5214 - precision: 0.

8/8 [==============================] - 0s 58ms/step - loss: 1.1549 - accuracy: 0.5093 - precision_1: 0.4688 - recall_1: 0.3505 - auc_1: 0.4935
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 1.1142 - accuracy: 0.5071 - precision_1: 0.4728 - recall_1: 0.4463 - auc_1: 0.5040
Epoch 7/50
8/8 [==============================] - 0s 54ms/step - loss: 1.0739 - accuracy: 0.5235 - precision_1: 0.4901 - recall_1: 0.4042 - auc_1: 0.5156
Epoch 8/50
8/8 [==============================] - 1s 64ms/step - loss: 1.0484 - accuracy: 0.5148 - precision_1: 0.4675 - recall_1: 0.2523 - auc_1: 0.4845
Epoch 9/50
8/8 [==============================] - 0s 60ms/step - loss: 1.0143 - accuracy: 0.4995 - precision_1: 0.4393 - recall_1: 0.2453 - auc_1: 0.4919
Epoch 10/50
8/8 [==============================] - 0s 60ms/step - loss: 0.9804 - accuracy: 0.5159 - precision_1: 0.4778 - recall_1: 0.3528 - auc_1: 0.5269
Epoch 11/50
8/8 [==============================] - 0s 60ms/step - loss: 0.9639 - accur

8/8 [==============================] - 1s 63ms/step - loss: 1.0397 - accuracy: 0.5060 - precision_2: 0.4661 - recall_2: 0.3692 - auc_2: 0.4947
Epoch 9/50
8/8 [==============================] - 0s 62ms/step - loss: 1.0084 - accuracy: 0.5082 - precision_2: 0.4658 - recall_2: 0.3341 - auc_2: 0.4934
Epoch 10/50
8/8 [==============================] - 0s 62ms/step - loss: 0.9806 - accuracy: 0.4973 - precision_2: 0.4424 - recall_2: 0.2780 - auc_2: 0.4955
Epoch 11/50
8/8 [==============================] - 0s 62ms/step - loss: 0.9568 - accuracy: 0.4995 - precision_2: 0.4436 - recall_2: 0.2664 - auc_2: 0.4875
Epoch 12/50
8/8 [==============================] - 0s 58ms/step - loss: 0.9338 - accuracy: 0.4918 - precision_2: 0.4437 - recall_2: 0.3318 - auc_2: 0.4840
Epoch 13/50
8/8 [==============================] - 1s 77ms/step - loss: 0.9084 - accuracy: 0.5159 - precision_2: 0.4770 - recall_2: 0.3388 - auc_2: 0.5138
Epoch 14/50
8/8 [==============================] - 0s 60ms/step - loss: 0.8873 - ac

8/8 [==============================] - 1s 70ms/step - loss: 0.9698 - accuracy: 0.5345 - precision_3: 0.5078 - recall_3: 0.2290 - auc_3: 0.5093
Epoch 11/50
8/8 [==============================] - 0s 62ms/step - loss: 0.9442 - accuracy: 0.5203 - precision_3: 0.4691 - recall_3: 0.1776 - auc_3: 0.5026
Epoch 12/50
8/8 [==============================] - 0s 54ms/step - loss: 0.9207 - accuracy: 0.5279 - precision_3: 0.4914 - recall_3: 0.2009 - auc_3: 0.5041
Epoch 13/50
8/8 [==============================] - 1s 66ms/step - loss: 0.9048 - accuracy: 0.5049 - precision_3: 0.4259 - recall_3: 0.1612 - auc_3: 0.4719
Epoch 14/50
8/8 [==============================] - 1s 77ms/step - loss: 0.8835 - accuracy: 0.5137 - precision_3: 0.4429 - recall_3: 0.1449 - auc_3: 0.4924: 0s - loss: 0.8858 - accuracy: 0.5052 - precision_3: 0.4390 - recall_3: 0.1479 - auc_3: 0.49
Epoch 15/50
8/8 [==============================] - 1s 67ms/step - loss: 0.8639 - accuracy: 0.5170 - precision_3: 0.4660 - recall_3: 0.2079 - auc

8/8 [==============================] - 0s 58ms/step - loss: 0.9270 - accuracy: 0.5027 - precision_4: 0.4430 - recall_4: 0.2360 - auc_4: 0.4917
Epoch 13/50
8/8 [==============================] - 0s 62ms/step - loss: 0.9082 - accuracy: 0.5257 - precision_4: 0.4847 - recall_4: 0.1846 - auc_4: 0.4786
Epoch 14/50
8/8 [==============================] - 0s 61ms/step - loss: 0.8827 - accuracy: 0.5455 - precision_4: 0.5520 - recall_4: 0.1612 - auc_4: 0.5149
Epoch 15/50
8/8 [==============================] - 1s 69ms/step - loss: 0.8645 - accuracy: 0.5301 - precision_4: 0.4952 - recall_4: 0.1215 - auc_4: 0.5363
Epoch 16/50
8/8 [==============================] - 0s 61ms/step - loss: 0.8524 - accuracy: 0.5104 - precision_4: 0.4596 - recall_4: 0.2523 - auc_4: 0.5001
Epoch 17/50
8/8 [==============================] - 0s 57ms/step - loss: 0.8367 - accuracy: 0.5181 - precision_4: 0.4793 - recall_4: 0.3248 - auc_4: 0.5227
Epoch 18/50
8/8 [==============================] - 0s 56ms/step - loss: 0.8244 - a

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3),input_shape=(None,n_length,n_features)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(50,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)
    
run_experiment_2(X,y)

Epoch 1/50
8/8 [==============================] - 0s 62ms/step - loss: 1.4031 - accuracy: 0.4962 - precision_5: 0.4688 - recall_5: 0.5607 - auc_5: 0.4676
Epoch 2/50
8/8 [==============================] - 1s 70ms/step - loss: 1.3110 - accuracy: 0.5203 - precision_5: 0.4871 - recall_5: 0.4416 - auc_5: 0.5185
Epoch 3/50
8/8 [==============================] - 1s 73ms/step - loss: 1.2601 - accuracy: 0.5356 - precision_5: 0.5074 - recall_5: 0.3224 - auc_5: 0.5309
Epoch 4/50
8/8 [==============================] - 0s 57ms/step - loss: 1.2221 - accuracy: 0.5126 - precision_5: 0.4745 - recall_5: 0.3692 - auc_5: 0.5065
Epoch 5/50
8/8 [==============================] - 1s 71ms/step - loss: 1.1817 - accuracy: 0.5093 - precision_5: 0.4741 - recall_5: 0.4276 - auc_5: 0.5033
Epoch 6/50
8/8 [==============================] - 1s 73ms/step - loss: 1.1463 - accuracy: 0.5038 - precision_5: 0.4539 - recall_5: 0.2874 - auc_5: 0.4937
Epoch 7/50
8/8 [==============================] - 1s 63ms/step - loss: 1.107

Epoch 1/50
8/8 [==============================] - 1s 67ms/step - loss: 1.3539 - accuracy: 0.4885 - precision_6: 0.4558 - recall_6: 0.4696 - auc_6: 0.5142
Epoch 2/50
8/8 [==============================] - 1s 68ms/step - loss: 1.2942 - accuracy: 0.5060 - precision_6: 0.4709 - recall_6: 0.4346 - auc_6: 0.5042
Epoch 3/50
8/8 [==============================] - 1s 67ms/step - loss: 1.2464 - accuracy: 0.5137 - precision_6: 0.4692 - recall_6: 0.2850 - auc_6: 0.4920
Epoch 4/50
8/8 [==============================] - 0s 61ms/step - loss: 1.1969 - accuracy: 0.4995 - precision_6: 0.4598 - recall_6: 0.3879 - auc_6: 0.4961
Epoch 5/50
8/8 [==============================] - 1s 68ms/step - loss: 1.1522 - accuracy: 0.5225 - precision_6: 0.4878 - recall_6: 0.3738 - auc_6: 0.5233
Epoch 6/50
8/8 [==============================] - 0s 62ms/step - loss: 1.1173 - accuracy: 0.5301 - precision_6: 0.4983 - recall_6: 0.3411 - auc_6: 0.5102
Epoch 7/50
8/8 [==============================] - 0s 62ms/step - loss: 1.091

Epoch 1/50
8/8 [==============================] - 1s 75ms/step - loss: 1.3784 - accuracy: 0.5027 - precision_7: 0.4747 - recall_7: 0.5701 - auc_7: 0.5168
Epoch 2/50
8/8 [==============================] - 0s 57ms/step - loss: 1.3080 - accuracy: 0.5071 - precision_7: 0.4604 - recall_7: 0.2991 - auc_7: 0.5280
Epoch 3/50
8/8 [==============================] - 1s 79ms/step - loss: 1.2519 - accuracy: 0.5137 - precision_7: 0.4770 - recall_7: 0.3879 - auc_7: 0.5412
Epoch 4/50
8/8 [==============================] - 1s 72ms/step - loss: 1.1970 - accuracy: 0.5444 - precision_7: 0.5168 - recall_7: 0.4322 - auc_7: 0.5733
Epoch 5/50
8/8 [==============================] - 0s 58ms/step - loss: 1.1665 - accuracy: 0.5444 - precision_7: 0.5294 - recall_7: 0.2523 - auc_7: 0.5563
Epoch 6/50
8/8 [==============================] - 0s 61ms/step - loss: 1.1108 - accuracy: 0.5608 - precision_7: 0.5408 - recall_7: 0.4182 - auc_7: 0.5971
Epoch 7/50
8/8 [==============================] - 1s 76ms/step - loss: 1.074

Epoch 1/50
8/8 [==============================] - 1s 73ms/step - loss: 1.4121 - accuracy: 0.4556 - precision_8: 0.4017 - recall_8: 0.3294 - auc_8: 0.4486
Epoch 2/50
8/8 [==============================] - 0s 58ms/step - loss: 1.3006 - accuracy: 0.5400 - precision_8: 0.5105 - recall_8: 0.4556 - auc_8: 0.5416
Epoch 3/50
8/8 [==============================] - 1s 75ms/step - loss: 1.2597 - accuracy: 0.5170 - precision_8: 0.4801 - recall_8: 0.3668 - auc_8: 0.5114
Epoch 4/50
8/8 [==============================] - 1s 72ms/step - loss: 1.2240 - accuracy: 0.4896 - precision_8: 0.4451 - recall_8: 0.3598 - auc_8: 0.4770
Epoch 5/50
8/8 [==============================] - 0s 61ms/step - loss: 1.1678 - accuracy: 0.5356 - precision_8: 0.5053 - recall_8: 0.4439 - auc_8: 0.5379
Epoch 6/50
8/8 [==============================] - 1s 77ms/step - loss: 1.1363 - accuracy: 0.5049 - precision_8: 0.4565 - recall_8: 0.2944 - auc_8: 0.5041
Epoch 7/50
8/8 [==============================] - 0s 61ms/step - loss: 1.100

Epoch 1/50
8/8 [==============================] - 1s 71ms/step - loss: 1.3655 - accuracy: 0.5235 - precision_9: 0.4923 - recall_9: 0.5257 - auc_9: 0.5308
Epoch 2/50
8/8 [==============================] - 1s 67ms/step - loss: 1.3207 - accuracy: 0.5016 - precision_9: 0.4703 - recall_9: 0.5000 - auc_9: 0.4979
Epoch 3/50
8/8 [==============================] - 1s 84ms/step - loss: 1.2506 - accuracy: 0.5115 - precision_9: 0.4746 - recall_9: 0.3925 - auc_9: 0.5365
Epoch 4/50
8/8 [==============================] - 1s 69ms/step - loss: 1.1907 - accuracy: 0.5487 - precision_9: 0.5296 - recall_9: 0.3341 - auc_9: 0.5869
Epoch 5/50
8/8 [==============================] - 1s 80ms/step - loss: 1.1465 - accuracy: 0.5553 - precision_9: 0.5251 - recall_9: 0.5374 - auc_9: 0.5933
Epoch 6/50
8/8 [==============================] - 1s 77ms/step - loss: 1.1139 - accuracy: 0.5641 - precision_9: 0.5295 - recall_9: 0.6285 - auc_9: 0.5976
Epoch 7/50
8/8 [==============================] - 1s 82ms/step - loss: 1.073

 Accuracy: 73.450% (+/-10.354) 
 Precision: 59.650% (+/-32.926) 
 Recall: 46.818% (+/-27.144) 
 AUC: 79.458% (+/-16.535) 


In [13]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='selu'))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'selu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)
    
run_experiment_3(X,y)

Epoch 1/50
8/8 [==============================] - 1s 71ms/step - loss: 1.5910 - accuracy: 0.4896 - precision_10: 0.4599 - recall_10: 0.5093 - auc_10: 0.4792
Epoch 2/50
8/8 [==============================] - 0s 62ms/step - loss: 1.4102 - accuracy: 0.5192 - precision_10: 0.4887 - recall_10: 0.5561 - auc_10: 0.5433
Epoch 3/50
8/8 [==============================] - 0s 60ms/step - loss: 1.4189 - accuracy: 0.5049 - precision_10: 0.4608 - recall_10: 0.3294 - auc_10: 0.5048
Epoch 4/50
8/8 [==============================] - 1s 64ms/step - loss: 1.3581 - accuracy: 0.5203 - precision_10: 0.4870 - recall_10: 0.4369 - auc_10: 0.5161
Epoch 5/50
8/8 [==============================] - 0s 62ms/step - loss: 1.2987 - accuracy: 0.5257 - precision_10: 0.4943 - recall_10: 0.5070 - auc_10: 0.5407
Epoch 6/50
8/8 [==============================] - 1s 79ms/step - loss: 1.2812 - accuracy: 0.5279 - precision_10: 0.4964 - recall_10: 0.4790 - auc_10: 0.5363
Epoch 7/50
8/8 [==============================] - 1s 78ms/

Epoch 1/50
8/8 [==============================] - 1s 70ms/step - loss: 1.6798 - accuracy: 0.4973 - precision_11: 0.4653 - recall_11: 0.4860 - auc_11: 0.4932
Epoch 2/50
8/8 [==============================] - 0s 58ms/step - loss: 1.5024 - accuracy: 0.5225 - precision_11: 0.4885 - recall_11: 0.3972 - auc_11: 0.5081
Epoch 3/50
8/8 [==============================] - 1s 70ms/step - loss: 1.4543 - accuracy: 0.4973 - precision_11: 0.4727 - recall_11: 0.6262 - auc_11: 0.4953
Epoch 4/50
8/8 [==============================] - 1s 64ms/step - loss: 1.4156 - accuracy: 0.5104 - precision_11: 0.4615 - recall_11: 0.2664 - auc_11: 0.5102
Epoch 5/50
8/8 [==============================] - 1s 66ms/step - loss: 1.3347 - accuracy: 0.5389 - precision_11: 0.5086 - recall_11: 0.4813 - auc_11: 0.5518
Epoch 6/50
8/8 [==============================] - 1s 70ms/step - loss: 1.2942 - accuracy: 0.5553 - precision_11: 0.5274 - recall_11: 0.4953 - auc_11: 0.5783
Epoch 7/50
8/8 [==============================] - 1s 66ms/

Epoch 1/50
8/8 [==============================] - 1s 63ms/step - loss: 1.5232 - accuracy: 0.5049 - precision_12: 0.4740 - recall_12: 0.5117 - auc_12: 0.5027
Epoch 2/50
8/8 [==============================] - 1s 65ms/step - loss: 1.4361 - accuracy: 0.5115 - precision_12: 0.4774 - recall_12: 0.4439 - auc_12: 0.4902
Epoch 3/50
8/8 [==============================] - 0s 58ms/step - loss: 1.3632 - accuracy: 0.5334 - precision_12: 0.5026 - recall_12: 0.4439 - auc_12: 0.5258 0s - loss: 1.3951 - accuracy: 0.5016 - precision_12: 0.4737 - recall_12: 0.4131 - auc_12: 
Epoch 4/50
8/8 [==============================] - 0s 57ms/step - loss: 1.3240 - accuracy: 0.5422 - precision_12: 0.5175 - recall_12: 0.3458 - auc_12: 0.5426
Epoch 5/50
8/8 [==============================] - 0s 61ms/step - loss: 1.3179 - accuracy: 0.5148 - precision_12: 0.4834 - recall_12: 0.5093 - auc_12: 0.5071
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 1.2951 - accuracy: 0.5016 - precision_12: 0.4646 - re

Epoch 1/50
8/8 [==============================] - 0s 56ms/step - loss: 1.5682 - accuracy: 0.5115 - precision_13: 0.4784 - recall_13: 0.4650 - auc_13: 0.5093
Epoch 2/50
8/8 [==============================] - 0s 57ms/step - loss: 1.4624 - accuracy: 0.4995 - precision_13: 0.4619 - recall_13: 0.4112 - auc_13: 0.5153
Epoch 3/50
8/8 [==============================] - 0s 51ms/step - loss: 1.3390 - accuracy: 0.5553 - precision_13: 0.5291 - recall_13: 0.4673 - auc_13: 0.5815
Epoch 4/50
8/8 [==============================] - 0s 56ms/step - loss: 1.3365 - accuracy: 0.5279 - precision_13: 0.4962 - recall_13: 0.4603 - auc_13: 0.5410
Epoch 5/50
8/8 [==============================] - 0s 58ms/step - loss: 1.2541 - accuracy: 0.5717 - precision_13: 0.5499 - recall_13: 0.4766 - auc_13: 0.6118
Epoch 6/50
8/8 [==============================] - 1s 64ms/step - loss: 1.2258 - accuracy: 0.6079 - precision_13: 0.5875 - recall_13: 0.5491 - auc_13: 0.6401
Epoch 7/50
8/8 [==============================] - 0s 59ms/

Epoch 1/50
8/8 [==============================] - 1s 75ms/step - loss: 1.5230 - accuracy: 0.5159 - precision_14: 0.4850 - recall_14: 0.5280 - auc_14: 0.5112
Epoch 2/50
8/8 [==============================] - 0s 61ms/step - loss: 1.4376 - accuracy: 0.5268 - precision_14: 0.4935 - recall_14: 0.3528 - auc_14: 0.5105
Epoch 3/50
8/8 [==============================] - 1s 67ms/step - loss: 1.3750 - accuracy: 0.5082 - precision_14: 0.4746 - recall_14: 0.4579 - auc_14: 0.5069
Epoch 4/50
8/8 [==============================] - 1s 69ms/step - loss: 1.3288 - accuracy: 0.5115 - precision_14: 0.4790 - recall_14: 0.4790 - auc_14: 0.5165
Epoch 5/50
8/8 [==============================] - 0s 59ms/step - loss: 1.2915 - accuracy: 0.5312 - precision_14: 0.5000 - recall_14: 0.3598 - auc_14: 0.5186
Epoch 6/50
8/8 [==============================] - 0s 62ms/step - loss: 1.2395 - accuracy: 0.5334 - precision_14: 0.5024 - recall_14: 0.4836 - auc_14: 0.5488
Epoch 7/50
8/8 [==============================] - 1s 65ms/

 Accuracy: 85.939% (+/-11.355) 
 Precision: 85.646% (+/-17.025) 
 Recall: 78.409% (+/-10.365) 
 AUC: 92.315% (+/-8.070) 
